## 8.2.1 为什么需要被管理的属性

In [2]:
# 我们可以把内置函数的结果赋值给一个类属性来创建一个property
class Person:
    def __init__(self, name):
        self._name = name
    def getName(self):
        print('fetch...')
        return self._name
    def setName(self,value):
        print('change...')
        self._name = value
    def delName(self):
        print('remove...')
        del self._name

    name = property(getName, setName, delName, "name property docs")

# 下面我们来演示一下
bob = Person("bot smith")
print(bob.name)
print(Person.name.__doc__)
bob.name = 'robert'
print(bob.name)
del bob.name

fetch...
bot smith
name property docs
change...
fetch...
robert
remove...


In [3]:
# 子类可以继承property
class APerson(Person):
    pass
a = APerson("123")
print(a.name)

fetch...
123


In [4]:
# property 还可以动态的计算属性
class ProSquare:
    def __init__(self, start):
        self.value = start
    def getX(self):
        return self.value**2
    def setX(self, value):
        self.value = value
    X = property(getX,setX)

P = ProSquare(3)
Q = ProSquare(32)

# 通过前面的定义，我们可以把它当成静态数据一样去访问
print(P.X)
P.X = 4
print(P.X)
print(Q.X)

9
16
1024


In [5]:
# 我们也可以通过装饰器来实现
class Person:
    def __init__(self, name):
        self._name = name
    @property
    def name(self):
        print('fetch...')
        return self._name
    @name.setter
    def name(self,value):
        print('change...')
        self._name = value
    @name.deleter
    def name(self):
        print('remove...')
        del self._name

# 这个和上面是一样的
bob = Person("bot smith")
print(bob.name)
print(Person.name.__doc__)
bob.name = 'robert'
print(bob.name)
del bob.name

fetch...
bot smith
None
change...
fetch...
robert
remove...


## 8.2.2 描述符

In [6]:
# 下面我们定义一个描述符
class Descriptor:
    def __get__(self, instance, owner):
        print(self,instance,owner,sep='\n')
# 有一个类继承了描述符
class Subject:
    attr = Descriptor()
x = Subject()
print(x.attr)

<class '__main__.Subject'>
None


In [7]:
print(Subject.attr)

None
<class '__main__.Subject'>
None


In [8]:
# 如果只有get方法那就是只读描述符
class D:
    def __get__(*args): print('get')
class C:
    a = D()
X = C()
X.a
C.a
X.a = 99
print(X.a)
print(X.__dict__.keys())

get
get
99
dict_keys(['a'])


In [9]:
Y = C()
Y.a
C.a

get
get


In [10]:
# 我们也可以这样，如果用户想对类进行赋值就抛出异常
class D:
    def __get__(*args): print('get')
    def __set__(*args):  raise AttributeError('cannot set')

class C:
    a = D()
X = C()
X.a

get


In [11]:
X.a = 99

AttributeError: cannot set

In [12]:
# 下面来一个实例
class Name:
    "name description"
    def __get__(self, instance, owner):
        print('fetch...')
        return instance._name
    def __set__(self, instance, value):
        print('change...')
        instance._name = value
    def __delete__(self, instance):
        print('remove...')
        del instance._name

class Person:
    def __init__(self, name):
        self._name = name
    name = Name()

bob = Person('bot smith')
print(bob.name)
bob.name = 'rebert'
print(bob.name)
del bob.name

fetch...
bot smith
change...
fetch...
rebert
remove...


In [ ]:
# 描述符类同样可以被子类继承，这里就不演示了

## 8.2.3 getattr和getattribute

![](.8_images/095a5fe1.png)

In [13]:
class Catcher:
    def __getattr__(self, name):
        print('get %s' % name)
    def __setattr__(self, key, value):
        print('set %s %s'%(key,value))
X = Catcher()
X.job
X.pay
X.pay = 99

get job
get pay
set pay 99


In [14]:
# 可以使用委托设计模式
class Wrapper:
    def __init__(self, object):
        self.wrapped = object
    def __getattr__(self, item):
        print('trace:'+item)
        return getattr(self.wrapped, item)

x = Wrapper([1,2,3])
# 当我们执行append时会触发getattr方法
x.append(4)
print(x.wrapped)

trace:append
[1, 2, 3, 4]


In [ ]:
# 不要在属性拦截方法里面再次获取属性，要不然会触发死循环
def __getattribute__(self,name):
    x = self.other

In [ ]:
# 我们可以把它指向一个父类
def __getattribute__(self,name):
    x = object.__getattribute__(self, 'other')

In [ ]:
# 设置属性同样会触发死循环，所以我们需要这样操作,当然也可以传给父类
def __setattr__(self, name,value):
    self.__dict__['other'] = value

In [ ]:
# 不能这样用，这样也会触发循环、
def __getattribute__(self,name):
    x = self.__dict__['other']

In [15]:
# 下面我们总结一下getattr和getattribute的区别
class getAttr:
    attr1 = 1
    def __init__(self):
        self.attr2 = 2
    def __getattr__(self, item):
        print('get'+item)
        if item == 'attr3':
            return 3
        else:
            raise AttributeError(item)
x = getAttr()
print(x.attr1)
print(x.attr2)
print(x.attr3)

1
2
getattr3
3


In [16]:
# __getattribute__可以拦截所有的
class getAttr(object):
    attr1 = 1
    def __init__(self):
        self.attr2 = 2
    def __getattribute__(self, item):
        print('get'+item)
        if item == 'attr3':
            return 3
        else:
            return object.__getattribute__(self,item)
x = getAttr()
print(x.attr1)
print(x.attr2)
print(x.attr3)

getattr1
1
getattr2
2
getattr3
3


## 8.2.4 属性验证